In [ ]:
def make_prediction(model_dict, dates, ticker_symbols):
    '''
    
    '''
    pred_base_range = model_dict["pred_base_range"]
    
    # Check if the model was trained for all the provided ticker symbols:
    ticker_symbols = set(ticker_symbols)
    tickers_to_remove = []
    for ticker in ticker_symbols:
        if ticker not in model_dict["ticker_symbols"]:
            print(f"WARN: model {model_dict['name']} was not trained to predict prices for {ticker}!")
            tickers_to_remove.append(ticker)
    
    for ticker in tickers_to_remove:
            ticker_symbols.remove(ticker)
            
    # Check if the date ranges are outside the range which were used for training:
    dates = set(dates)
    dates_to_remove = []
    for date in dates:
        if (date > model_dict["start_date"]) and (date < model_dict["end_date"]):
            print(f"WARN: model {model_dict['name']}  was trained with the date {date} and therefore no predictions for that date can be made!")
            dates_to_remove.append(date)
    for date in dates_to_remove:
        dates.remove(date)
            
    # Initialize a result dictionary
    result = {}
    for ticker in ticker_symbols:
        result[ticker] = {}
        # sort dates so there is no need to predict a day twice
        for target_date in dates:
            print(f"Predicting {ticker} price for {target_date}") 
            start_date = datetime.today() - timedelta(days=pred_base_range)
            # if the date is to far in the future, multiple iterations are necessary
            
            delta = target_date - datetime.today() # as timedelta to predict

            # If no Adj close is available for today, today also needs to be predicted
            for i in range(1, delta.days + 2):
                day = datetime.today() + timedelta(days=i)
                print(day)
            
            
            available_data = get_stock_data(ticker_symbols = [ticker], start_date =  start_date, end_date = datetime.today(), date_index = True, columns="all")
            formated_data = tickers_as_columns(available_data)
            filtered_data = formated_data.filter([ticker])
            dataset = filtered_data.values
            scaler, y_test = scale_data(dataset)
            print(y_test.shape)
            
            prediction = 0
            result[ticker][target_date] = prediction
        
    return result

In [ ]:
# This takes a subset of the dataframe which is still a dataframe with 2D (len,1)
filtered_data = formated_data.filter(['GOOG'])
# I assume this creates a Pandas Series object which is just 1D
# close_data = formated_data['GOOG']
# 2. Convert the data into array for easy evaluation
dataset = filtered_data.values

pred_base_range = 60

scaler, scaled_dataset = scale_data(dataset)
x_train, y_train, x_test, y_test, training_data_len = train_test_split(scaled_dataset, days = pred_base_range)

In [ ]:
def tickers_as_columns(data_dict, column="Adj Close"):
        '''
        This method reformats a dictionary generated by the "get_stock_data"-method.
        The structure before is the following
        dict{
                stock1: [[High], [Low], [Close] ...],
                stock2: [[High], [Low], [Close] ...],
                ...
        }

        The method creates a dataframe which keeps the dates as the index, but adds a column for each stock based on a chosen value. The default value is "Adj Close".
        '''
        columns = []
        column_names = []
        for key in data_dict:
            column_names.append(key)
            columns.append(data_dict[key][column])
        df = pd.DataFrame(columns, index = column_names)
        return df.T

In [ ]:


# def plot_approximation(filtered_data, training_data_len, predictions):
#     train = filtered_data[:training_data_len]
#     valid = filtered_data[training_data_len:].copy()
#     valid.loc[:,'Predictions'] = predictions

#     plt.title('Model')
#     plt.xlabel('Date')
#     plt.ylabel('Close')

#     plt.plot(train['GOOG'])
#     plt.plot(valid[['GOOG', 'Predictions']])

#     plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')

#     plt.show()

In [ ]:
# def load_model_dict(model_name, model_dir = "./models/"):
#     with open(model_dir + model_name + "_metadata.obj", 'rb') as file:
#         # Call load method to deserialze
#         model_dict = pickle.load(file)

#     model_dict["model"] = load_model(model_name, model_dir)
#     return model_dict

# def load_model(model_name, model_dir = "./models/"):
#     model = tf.keras.models.load_model(model_dir + model_name + "_model")
#     return model

# def store_model_pickle(name, model, model_dir = "./models/"):
#     try:
#         model_params = model.get_params()
#         fileObj = open(f'{model_dir}{name}_params.obj', 'wb')
#         pickle.dump(model_params,fileObj)
#         fileObj.close()
#     except:
#         pass
#     fileObj = open(f'{name}.obj', 'wb')
#     pickle.dump(model,fileObj)
#     fileObj.close()
    
# def store_model_keras(name, model, model_dir = "./models/"):
#     model.save(f'{model_dir}{name}', save_format="h5" )  # creates a HDF5 file 'my_model.h5'


#     def save_model_dict(model_dict, model_dir = "./models/"):
#         model = model_dict["model"]
#         # First store the model using the Keras function because the model cannot be stored with pickle (weakref-error)
#         store_model_keras(model_dict["name"] + "_model", model)
#         model_dict["model"] = None
#         fileObj = open(model_dir + model_dict["name"] + "_metadata.obj", 'wb')
#         pickle.dump(model_dict,fileObj)
#         model_dict["model"] = model

#         fileObj.close()
        

In [ ]:
def setup_model(ticker, name, start_date, end_date = datetime.today(), pred_base_range = 60):
    # Create dictionary to store the model and its metadata
    model_dict = {}
    model_dict["name"] = name
    model_dict["ticker"] = ticker
    model_dict["start_date"] = start_date
    model_dict["end_date"] = end_date
    model_dict["pred_base_range"] = pred_base_range
    
    # Get the data, rescale it and divide it into training and test set
    df = get_adj_close_df([ticker], start_date, end_date = datetime.today())
    df_values = df.values
    scaler, scaled_dataset = scale_data(df_values)
    x_train, y_train, x_test, y_test, training_data_len = train_test_split(scaled_dataset, days = pred_base_range)
    
    # Train a model with the data
    
    
    model = get_model(input_shape = x_train.shape[1])
    model.fit(x_train, y_train, batch_size=1, epochs=1)
    model_dict["model"] = model
    
    return model_dict
    #store_model_keras("first_lstm", first_lstm)

In [ ]:
def create_timerange_from_now(include_today = True, n_days = 5, weekends = False):
    '''
    Returns a list of all future dates from now
    '''
    day_list = list()
    if include_today:
        i = 0
    else:
        i = 1
    while len(day_list) < n_days:
        next_day = datetime.now() + timedelta(days=i)
        i = i + 1
        # Consider if the day is on the weekend
        if not weekends:
            if next_day.weekday() < 5:
                day_list.append(next_day)
        else:
            day_list.append(next_day)
    return day_list

def extend_df(df, predictions, new_timerange = None):
    '''
    This methods takes a df of values and appends the predicted values to the dataframe. The new timerange is used as index.
    '''
    column_name = df.columns[0]
    if new_timerange == None:
        new_timerange = create_timerange_from_now()
    date_index = pd.DatetimeIndex(new_timerange).normalize()
    new_df = pd.DataFrame(data = predictions, index = date_index, columns = [column_name])
    combined_df = pd.concat([df, new_df])
    return combined_df        